# Data Acquisition

## Connect to Cloud Firestore

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('./serviceAccount.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

/Users/kooler/Project/side-project/dark-traveller-24-analysis/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Retrieve collection

### Users

In [2]:
users_ref = db.collection("users")
docs = users_ref.stream()

users = {}

for doc in docs:
    users[doc.id] = doc.to_dict()

I0000 00:00:1722702007.422288 8209835 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [3]:
users

{'sd1-1@example.com': {'capturedPoints': [{'userId': 'sd1-1@example.com',
    'userName': '南區中心1隊',
    'expiredAt': '2024-08-02T22:35:06.177Z',
    'createdAt': '2024-08-02T22:03:00.057Z',
    'level': 3,
    'pointId': 'Un4smPc0lSA8jpfvA5zJ',
    'pointName': '天水圍游泳池'},
   {'userId': 'sd1-1@example.com',
    'userName': '南區中心1隊',
    'expiredAt': '2024-08-02T22:33:31.782Z',
    'createdAt': '2024-08-02T22:06:50.535Z',
    'level': 3,
    'pointId': '0gHiSX9dMHkVpffFR9RU',
    'pointName': '天水圍體育館'}],
  'level': 1,
  'email': 'sd1-1@example.com',
  'upgradedPoints': ['1', '2'],
  'name': '南區中心1隊'},
 'sd1-2@example.com': {'upgradedPoints': [],
  'level': 1,
  'email': 'sd1-2@example.com',
  'capturedPoints': [],
  'name': '南區中心2隊'},
 'skc-1@example.com': {'capturedPoints': [{'userId': 'skc-1@example.com',
    'userName': '南葵中心1隊',
    'expiredAt': '2024-08-02T21:07:15.858Z',
    'createdAt': '2024-08-02T20:22:21.223Z',
    'level': 1,
    'pointId': 'xFldNrcu765eRZc2hrsn',
    'pointNa

### Points

In [4]:
points_ref = db.collection("points")
docs = points_ref.stream()

points = {}

for doc in docs:
    points[doc.id] = doc.to_dict()

In [5]:
points

{'0gHiSX9dMHkVpffFR9RU': {'point': '天水圍體育館',
  'description': '',
  'isPublic': True,
  'heroImage': 'https://firebasestorage.googleapis.com/v0/b/dark-traveller-24-dev.appspot.com/o/%E5%A4%A9%E6%B0%B4%E5%9C%8D%E9%AB%94%E8%82%B2%E9%A4%A8.jpeg?alt=media&token=537bfc9d-e61b-4eb3-9c71-44ed09f90fe1',
  'location': {'long': 114.0067398, 'lat': 22.4548219}},
 '1ZflGseWvmvEMDtj6YOq': {'point': '隱藏集合點',
  'description': '',
  'isPublic': True,
  'heroImage': '',
  'location': {'long': 114.0085632, 'lat': 22.4510973}},
 '2d52tp09iWfccGggsMH5': {'point': '流浮山消防局',
  'description': '',
  'isPublic': True,
  'heroImage': 'https://firebasestorage.googleapis.com/v0/b/dark-traveller-24-dev.appspot.com/o/%E6%B5%81%E6%B5%AE%E5%B1%B1%E6%B6%88%E9%98%B2%E5%B1%80.jpeg?alt=media&token=73630649-9903-4056-93b3-3a0e08c9d2b1',
  'location': {'long': 114.0022616, 'lat': 22.4713162}},
 '2wd13TbtnVTB8OIHhRa4': {'point': '天水圍運動場',
  'description': '',
  'isPublic': True,
  'heroImage': 'https://firebasestorage.googl

### Activity log

In [6]:
activity_logs_ref = db.collection("activitylog")
docs = activity_logs_ref.stream()

activity_logs = {}

for doc in docs:
    activity_logs[doc.id] = doc.to_dict()

## View Data

In [7]:
import json

In [8]:
with open('./data/users.json', 'w') as json_file:
    json.dump(users, json_file)

In [9]:
with open('./data/points.json', 'w') as json_file:
    json.dump(points, json_file)

In [10]:
from google.cloud.firestore_v1._helpers import DatetimeWithNanoseconds
from datetime import datetime

def firestore_to_datetime(firestore_datetime):
    if isinstance(firestore_datetime, DatetimeWithNanoseconds):
        return datetime(
            firestore_datetime.year,
            firestore_datetime.month,
            firestore_datetime.day,
            firestore_datetime.hour,
            firestore_datetime.minute,
            firestore_datetime.second,
            firestore_datetime.microsecond
        )
    else:
        raise TypeError("The provided object is not of type DatetimeWithNanoseconds")

In [11]:
def json_serializer(obj):
    if isinstance(obj, DatetimeWithNanoseconds):
        obj = firestore_to_datetime(obj)
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError(f'Type {type(obj)} not serializable')

with open('./data/activity_logs.json', 'w') as json_file:
    json.dump(activity_logs, json_file, default=json_serializer)